# Script to sign a predefined CSR
This script creates an EC key pair and signs a predefined Certificated Signing Request (CSR) of SCMS.
The CSR is also known as enrollment request in SCMS. The signature uses ECDSA and SECP256R1 elliptic curve according to IEEE 1609.2 standard.

## Import libraries

In [83]:
from IPython.core.display import display, HTML
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec, utils
from  cryptography.hazmat.primitives import serialization

## Programmatically create the CSR (predefined)

Create the CSR programmatically (the tbsRequest part of the CSR is already figured out using ASN.1 studio)
and add public key (byte offset is known ahead of time). The predefined CSR data (tbsRequest field specifically) is defined as follows.

In [84]:
tbsRequest = "018180000160D29D484481057273655F31000000000460D29D488600028301018003480101E08001030001820003204095000320409701000100808080"

## Create ECC key pair

In [85]:
private_key = ec.generate_private_key(
    ec.SECP256R1()
)
public_key = private_key.public_key()


## Add public key in the compressed format
Note that the public key compressed format is "0x02/0x03 + x-point", so you have to cut off the first byte

In [86]:
# compress the public key
public_key_bytes = public_key.public_bytes(serialization.Encoding.X962, serialization.PublicFormat.CompressedPoint)
public_key_hex = bytes.hex(public_key_bytes)
typeStr = public_key_hex[:2]
public_key_hex = public_key_hex[2:]
print("type: ", typeStr, ", public key: ", public_key_hex)
public_key_part = ("82" if typeStr == "02" else "83") + public_key_hex
print(public_key_part)
tbsRequest = tbsRequest + public_key_part

type:  02 , public key:  309e49889406da9d903caa89026708bc71230a9eb66649869025a3201cd4b4f9
82309e49889406da9d903caa89026708bc71230a9eb66649869025a3201cd4b4f9


## Sign the data (tbsRequest in the CSR)
Note that sometimes the r and s values are not 64 hex characters or 32 bytes,
sometimes become 63 bytes, may need to add padding

In [87]:
data = bytes.fromhex(tbsRequest)
signature = private_key.sign(
     data,
     ec.ECDSA(hashes.SHA256())
 )
r, s = utils.decode_dss_signature(signature)
rStr = "{:X}".format(r)
sStr = "{:X}".format(s)

## Add the signature to the CSR

In [88]:
signedCSR = "038381A200" + tbsRequest + "828080" + rStr + sStr

Perform quick verify to ensure signature is working

In [89]:
public_key.verify(signature, data, ec.ECDSA(hashes.SHA256()))
print("Quick verify: successful!")

Quick verify: successful!


## Output the results as hex encoded strings
We will test two tests, first is to verify signature using the ouputted data file and signature file.
Second is to verify signature using only the CSR.

In [90]:
serialized_public = public_key.public_bytes(
     encoding=serialization.Encoding.PEM,
     format=serialization.PublicFormat.SubjectPublicKeyInfo
)
print("public key (PEM format):")
for l in serialized_public.splitlines():
    print(l.decode('utf-8'))

print("\npublic key (DER compressed format):", public_key_hex)
print(f'public key type ({typeStr}): {"compressed-y-1" if typeStr == "03" else "compressed-y-0"}')
print(f"[data:{len(tbsRequest)}]:", tbsRequest)
print(f"[sig:{len(signature)}]:{signature.hex()}")
print(f"[r:{len(rStr)}]:{rStr}")
print(f"[s:{len(sStr)}]:{sStr}")
print(f"[signed-CSR:{len(signedCSR)}]: ", signedCSR)

public key (PEM format):
-----BEGIN PUBLIC KEY-----
MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEMJ5JiJQG2p2QPKqJAmcIvHEjCp62
ZkmGkCWjIBzUtPnArUoCyA4zrnQ0Y5giHpNtOgNPZLqHLfkwDjzGyBcX0A==
-----END PUBLIC KEY-----

public key (DER compressed format): 309e49889406da9d903caa89026708bc71230a9eb66649869025a3201cd4b4f9
public key type (02): compressed-y-0
[data:188]: 018180000160D29D484481057273655F31000000000460D29D488600028301018003480101E0800103000182000320409500032040970100010080808082309e49889406da9d903caa89026708bc71230a9eb66649869025a3201cd4b4f9
[sig:71]:3045022100e74fe3dfc871a7dcb85c48f003dd87ec89bde939dcf0fd41b16f7dc619ff451502205d275dc37fdba78ed5c3c8476f759beca8a9f814cc95c68abc000832bf263aab
[r:64]:E74FE3DFC871A7DCB85C48F003DD87EC89BDE939DCF0FD41B16F7DC619FF4515
[s:64]:5D275DC37FDBA78ED5C3C8476F759BECA8A9F814CC95C68ABC000832BF263AAB
[signed-CSR:332]:  038381A200018180000160D29D484481057273655F31000000000460D29D488600028301018003480101E0800103000182000320409500032040970100010080808082309e4988